In [15]:
import sys
from pathlib import Path

# Đường dẫn tới thư mục chứa module export_manga
module_path = Path('../export_manga.py').resolve()

# Thêm đường dẫn tới sys.path
sys.path.append(str(module_path.parent))

# Import module
import export_manga

# Sử dụng hàm export_mangas_to_dict từ module export_manga
mangas_df = export_manga.export_mangas_to_dict()
print(mangas_df)


Ngắt kết nối với MongoDB
                         _id  \
0   664f7bbff7d334bb0ba94e00   
1   66648d1fe14004ab2684b308   
2   66648eb0e14004ab2684b3c3   
3   66648f04e14004ab2684b3e4   
4   66649528e14004ab2684b790   
5   666495a0e14004ab2684b890   
6   66649682e14004ab2684b90c   
7   667098f93ceb1e17489672d0   
8   667868b8c070b2f1253fb148   
9   6678698fc070b2f1253fb1d2   
10  66786bccc070b2f1253fb266   
11  66786c78c070b2f1253fb2ad   
12  667873223f6da702146853d2   
13  667875ef07cebf7337495a38   
14  6678783207cebf7337495d20   
15  6678797c07cebf7337495ef1   
16  66787a1407cebf7337495f25   
17  66787ace07cebf7337495f6d   
18  66787e4607cebf7337495fe3   
19  66787ec307cebf7337496017   
20  66787fc607cebf7337496095   
21  6678806007cebf73374960c3   
22  6678836f07cebf7337496332   
23  667883f207cebf7337496366   

                                                 name  \
0                                              Naruto   
1                                           ONE PIECE   
2  

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [17]:
tfidf = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9\-]+')

In [18]:
vl = mangas_df['genre'][:4]

In [19]:
mangas_df['genre'][:4]

0                                     action|adventure
1    action|adventure|comedy|drama|fantasy|shounen|...
2                             action|adventure|shounen
3                     action|adventure|fantasy|shounen
Name: genre, dtype: object

In [20]:
tfidf_matrix = tfidf.fit_transform(mangas_df['genre'])

In [21]:
import pandas as pd

In [22]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
print(tfidf_df)

      action  adventure  chuyen-sinh    comedy     drama    ecchi   fantasy  \
0   0.615742   0.787948     0.000000  0.000000  0.000000  0.00000  0.000000   
1   0.313918   0.401712     0.000000  0.380776  0.380776  0.00000  0.425116   
2   0.517532   0.662272     0.000000  0.000000  0.000000  0.00000  0.000000   
3   0.423808   0.542336     0.000000  0.000000  0.000000  0.00000  0.573933   
4   0.383988   0.000000     0.000000  0.465770  0.000000  0.00000  0.000000   
5   0.517532   0.000000     0.000000  0.000000  0.000000  0.00000  0.000000   
6   0.344629   0.000000     0.000000  0.418029  0.000000  0.00000  0.000000   
7   0.000000   0.000000     0.000000  0.000000  0.327760  0.00000  0.000000   
8   0.000000   0.000000     0.000000  0.000000  0.000000  0.00000  0.000000   
9   0.000000   0.000000     0.000000  0.000000  0.362192  0.00000  0.404367   
10  0.000000   0.000000     0.000000  0.707107  0.707107  0.00000  0.000000   
11  0.000000   0.000000     0.000000  0.291290  0.00

In [23]:
tfidf_matrix

<24x25 sparse matrix of type '<class 'numpy.float64'>'
	with 106 stored elements in Compressed Sparse Row format>

In [24]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [25]:
cosine_sim

array([[1.        , 0.50982022, 0.84050169, 0.68828885, 0.23643726,
        0.31866598, 0.21220263, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.38758281, 0.23374351,
        0.        , 0.        , 0.        , 0.40343549, 0.2517799 ,
        0.18465024, 0.26095634, 0.26494172, 0.26095634],
       [0.50982022, 1.        , 0.60656655, 0.74070678, 0.4300093 ,
        0.60656655, 0.56309378, 0.26370742, 0.        , 0.41255186,
        0.53849889, 0.1109164 , 0.53849889, 0.52732667, 0.22623695,
        0.        , 0.46669535, 0.29192915, 0.32041476, 0.3426378 ,
        0.40529015, 0.74070678, 0.48185048, 0.49671881],
       [0.84050169, 0.60656655, 1.        , 0.8189024 , 0.41653326,
        0.5613962 , 0.37383893, 0.        , 0.        , 0.16937116,
        0.        , 0.        , 0.        , 0.46113269, 0.19646181,
        0.        , 0.        , 0.        , 0.33908821, 0.35325936,
        0.15519884, 0.4597287 , 0.46674978, 0.4597287 ],
       [0.688

In [26]:
import pandas as pd

In [27]:
def get_recommendations_by_manga_ids(manga_ids, cosine_sim=cosine_sim, mangas_df=mangas_df):
    recommended_mangas = pd.DataFrame(columns=['name', 'author', 'genre'])

    for manga_id in manga_ids:
        idx = mangas_df.index[mangas_df['_id'] == manga_id].tolist()

        if idx:
            idx = idx[0]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
            manga_indices = [i[0] for i in sim_scores]
            recommended_mangas = pd.concat([recommended_mangas, mangas_df.iloc[manga_indices]], ignore_index=True)

    return recommended_mangas

In [28]:
mangaId = ['66786c78c070b2f1253fb2ad']
result = get_recommendations_by_manga_ids(mangaId)
result = result['_id'].tolist()
result


['66648eb0e14004ab2684b3c3',
 '66648f04e14004ab2684b3e4',
 '66648d1fe14004ab2684b308',
 '66787e4607cebf7337495fe3',
 '667875ef07cebf7337495a38',
 '666495a0e14004ab2684b890',
 '6678836f07cebf7337496332',
 '6678806007cebf73374960c3',
 '667883f207cebf7337496366',
 '66787ec307cebf7337496017']